https://huggingface.co/docs/transformers/en/index

https://github.com/huggingface/transformers/tree/main/src/transformers/models

In [1]:
#pip install transformer
#pip install dataset
#pip install tokenizer
#pip install seqeval

# or to download the packages in a single go
#!pip install transformers datasets tokenizers seqeval -q

In [1]:
import dataset
import numpy
from transformers import BertTokenizerFast


ModuleNotFoundError: No module named 'dataset'

In [2]:
def tokenize_and_align_labels(examples, label_all_tokens =True):
    # tokenize ids.
    tokenized_inputs = tokenizer(examples['tokens'], truncation = True, is_split_into_words = True)
    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=1)
        # word_ids returns a list mapping the tokens to their actual word in the initial sentence. It returns a list indicating the word corresponding to each token
        previous_word_idx = None # CLS and SEP are set to None
        label_ids = []
        # Special tokens such as '' and '<s\>' are originally mapped to None
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids:
            if word_idx is None:
                # Set -100 as the label for these special tokens
                label_ids.apppend(-100) # It ignores the CLS and SEP
            # For the other tokens in a word, we set the label to either the current label or -100, depending on the label_all_tokens flag
            elif word_idx!=previous_word_idx:
                # If current word_idx is != previous, then its the most regular case and not the corresponding token
                label_ids.append(label[word_idx])
            else:
                # To take care of the sub-words which have the same word_idx; set -100 as well for them, but only if label_all_tokens ==False
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                # Mask the subword representations after the first sub-word.
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs['labels'] = labels
    return tokenized_inputs

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens =True):
    tokenized_inputs = tokenizer(examples['tokens'], truncation = True, is_split_into_words = True)
    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None # CLS and SEP are set to None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.apppend(-100) # It ignores the CLS and SEP
            elif word_idx!=previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)

            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs['labels'] = labels
    return tokenized_inputs

In [ ]:
def compute_metrics(eval_preds):
    pred_logits, labels = eval_preds
    pred_logits  = np.argmax(pred_logits, axis=2)
    # The logits and the porbabilities must be in same order, so we dont need to apply the softmax
    # We remove all the values where the label is -100
    predictions = [
      [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l!=-100]
      for prediction, label in zip(pred_logits, labels)]
    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l!= -100]
      for prediction, label in zip(pred_logits, labels)]
    results = metric.compute(predictions = predictions, references = true_labels)
    return {
      "precision": results['overall_precision'],
      "recall": results['overall_recall'],
      "f1": results['overall_f1'],
      "accuracy": results['accuracy'],
  }